In [1]:
import pandas as pd
import numpy as np
import unidecode
import re

In [2]:
# importar os dados csv para pandas dataframe
cvd_2020_1 = pd.read_csv('dados/2020_1.csv',delimiter=';')
cvd_2020_2 = pd.read_csv('dados/2020_2.csv',delimiter=';')
cvd_2021_1 = pd.read_csv('dados/2020_2.csv',delimiter=';')
cvd_2021_2 = pd.read_csv('dados/2020_2.csv',delimiter=';')
cvd_2022_1 = pd.read_csv('dados/2020_2.csv',delimiter=';')

#### Explorando e Tratando os Dados da Covid do período 2020.1

In [3]:
# identificar porcentagem dos valores propriamente categorizados como NaN para cada coluna
for collumn in list(cvd_2020_1.columns):
    porcentagem = ((cvd_2020_1[collumn].isna().sum()/len(cvd_2020_1[collumn]))*100)
    print(collumn,": ",porcentagem)

regiao :  0.0
estado :  0.0
municipio :  0.0
coduf :  0.0
codmun :  0.0
codRegiaoSaude :  0.0
nomeRegiaoSaude :  0.0
data :  0.0
semanaEpi :  0.0
populacaoTCU2019 :  0.0
casosAcumulado :  0.0
casosNovos :  0.0
obitosAcumulado :  0.0
obitosNovos :  0.0
Recuperadosnovos :  0.0
emAcompanhamentoNovos :  0.0
interior/metropolitana :  1.0045230125054578


**Foi identificado valores faltantes na variável 'interior/metropolitana', sendo esses preenchidos com valor NaN. Para resolver isso será realizado imputação de valores com base na variável município, ou seja se a cidade localizada na variável for do interior a variável 'interior/metropolitana' receberá 0 senão receberá 1. Abaixo segue a lista das cidades metrópoles mapeadas visando identificar no dataset as mesmas, sendo que para isso será feita transformações no dado em relação a formatação das strings**

In [4]:
metropolitanas = ['Goiânia','Cuiabá','Macapá','Manaus','Belém','Santarém','Porto Velho','Boa Vista',
                  'Caracaraí','Caroebe','Palmas','Gurupi','Vitória','Belo Horizonte','Rio de Janeiro',
                  'Santos','Campinas','Jundiaí','Piracicaba','Ribeirão Preto','São José do Rio Preto',
                  'São Paulo','Sorocaba','São José dos Campos','Arapiraca','São Miguel dos Campos',
                  'Maceió','Santana do Ipanema','Palmeira dos Índios','Penedo','Delmiro Gouveia',
                  'Viçosa','União dos Palmares','Feira de Santana','Salvador','Juazeiro do Norte',
                  'Fortaleza','Sobral','São Luís','Imperatriz','Araruna','Barra de Santa Rosa',
                  'Cajazeiras','Campina Grande','Esperança','Guarabira','Itabaiana','João Pessoa',
                  'Patos','Sousa','Mamanguape','Piancó','Recife','Natal','Aracaju','Apucarana',
                  'Campo Mourão','Cascavel','Curitiba','Londrina','Maringá','	Toledo','Umuarama',
                  'Rio do Sul','Criciúma','Chapecó','Joaçaba','São Miguel do Oeste','Florianópolis',
                  'Itajaí','Lages','Joinville','Tubarão','Blumenau','Porto Alegre','Caxias do Sul']
for value in range(len(metropolitanas)):
    metropolitanas[value] = re.sub('\W+','',unidecode.unidecode(metropolitanas[value].lower()))

**Abaixo será verificado qual valor binário da variável 'interior/metropolitana' está associado a cidades metropolitanas, para isso utilizamos uma amostra do dataset somente de cidades metropolitanas selecionadas, se a divisão da soma dos valores da viaríavel 'interior/metropolitana' pelo tamanho da amostra for igual a 1, iremos inferir 1 a todos os valores onde o município se localize em zona metropolitana**

In [28]:
# identificamos que 1 é para metrópoles e 0 para cidades do interior
count_metropolis = cvd_2020_1[(cvd_2020_1['municipio'] == 'Salvador') | (cvd_2020_1['municipio'] == 'São Paulo') | (cvd_2020_1['municipio'] == 'Goiânia')]
len(count_metropolis['interior/metropolitana']) == count_metropolis['interior/metropolitana'].sum()

True

**Aqui será criado um novo atributo com base na variável 'municipio', removendo a ascentuação, removendo espaços e transoformando os caracteres para minúsculos, pois iremos fazer algumas comparações com a lista de metrópoles acima para imputação dos valores faltantes na variável 'interior/metropolitana'**

In [5]:
cvd_2020_1['temp_municipio'] = cvd_2020_1['municipio'].str.lower()
cvd_2020_1['temp_municipio'] = [re.sub('\W+','',unidecode.unidecode(value)) for value in cvd_2020_1['temp_municipio']] 

**Aqui de fato fazemos a imputação nos valores faltantes levando em consideração o tipo de município e a lista que foi criada com as nomenclaturas das metrópoles**

In [ ]:
cvd_2020_1['interior/metropolitana'] = np.where((np.isnan(cvd_2020_1['interior/metropolitana'])) & ~(cvd_2020_1['temp_municipio'].isin(metropolitanas)),0,cvd_2020_1['interior/metropolitana'])
cvd_2020_1['interior/metropolitana'] = np.where((np.isnan(cvd_2020_1['interior/metropolitana'])) & (cvd_2020_1['temp_municipio'].isin(metropolitanas)),1,cvd_2020_1['interior/metropolitana'])